<a href="https://colab.research.google.com/github/AppleTater/Jinkerson-Lab-Research/blob/master/Cumulative.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Files that need to be uploaded


---
* TAGData.csv
* large-lib_full_2016.10.25.csv




# Importing Libraries


**Python Libraries**


In [ ]:
import pandas as pd                                       # importing common pandas libraries under the abbreviation 'pd' 
import matplotlib.pyplot as plt                           # was for the requested histogram under the abbreviation 'plt' 

import numpy.matlib                                       # to calculate mean and median
import numpy as np                                        # numpy under the abbreviation 'np' 
from numpy import median                                  # from the numpy library, import how to get the median

import scipy.stats as stats                               # libraries used for BH adjustment under the abbreviation 'stats' 
import scipy.special                                      # special libraries from scipy

import re                                                 # more libraries
import sys                                                # more libraries

from IPython.display import (                             # display data, if wanted, as different media file types
    display_pretty, display_html, display_jpeg,
    display_png, display_json, display_latex, display_svg
)

from IPython.display import display                       # more display tools
import itertools                                          # more display tools
import gc                                                 # more display tools

import statsmodels.api as sm                              # stats models under the abbreviation 'sm'
import statsmodels.stats.multitest as smm                 # stats models under the abbreviation 'smm'
import statsmodels.formula.api as smf                     # stats models under the abbreviation 'smf'

**R Libraries**

In [ ]:
%reload_ext rpy2.ipython

# %%R
# library(ggplot2)

import rpy2.robjects as robjects                          # common rpy2 objects under the abbreviation 'robjects'
from rpy2.robjects.packages import importr                # rpy2 packages under the abbreviation 'importr'
from rpy2.robjects.vectors import FloatVector             # rpy2 for float conversion
import gc                                                 # rpy2 library
R_stats = importr('stats')                                # rpy2 stats

from rpy2.rinterface_lib.sexp import StrSexpVector        # rpy2 interface
from functools import partial                             # rpy2 tools under the abbreviation 'partial'
from rpy2.ipython import html                             # html support
html.html_rdataframe=partial(html.html_rdataframe, table_class="docutils")

from rpy2.robjects.conversion import localconverter       # rpy2 converter under the abbreviation 'localconverter'  

import pandas as pd                                       # these libraries are for redundancy
import rpy2.robjects as ro                                # ↓
from rpy2.robjects.packages import importr                # ↓
from rpy2.robjects import pandas2ri                       # ↓

# Making of the Merged File



In [ ]:
data_df = pd.read_csv('TAG Data.csv')                           # read TAG Data.csv file, read in as 'data'

data_df = data_df[['gene' , 'R5-S#11-HLG_WHOLE_normalized_reads' , 'R5-S#11-ALL_WHOLE_normalized_reads']]       
      # create dataframe from the TAG Data using the included columns
data_df ['Ratios: HLG / ALL (Whole)'] = data_df ['R5-S#11-HLG_WHOLE_normalized_reads'] / data_df['R5-S#11-ALL_WHOLE_normalized_reads']       
      # create another column for the ratio, high lipid gate vs all

data_df = data_df.sort_values('gene')                           # sort 'data' by the gene names, alphabetically (numerically in this case)
data_df = data_df.replace([np.inf, -np.inf], np.nan)            # replace gene names that are 'np.inf' or '-np.inf' to 'np.nan'
data_df = data_df.dropna()                                      # drop all rows that have 'np.nan' which we coverted in the step above

data_df = data_df.drop(['R5-S#11-HLG_WHOLE_normalized_reads' , 'R5-S#11-ALL_WHOLE_normalized_reads'], axis = 1)
      # drop these columns because the only use for them was to create the ratio, so they are no longer needed

data_df = pd.DataFrame(data_df)
      # consolidate all changes made to 'data' to a dataframe

mean_df = data_df.groupby(['gene']).mean()                      # calculate the mean as an independent dataframe
mean_df.rename(columns = {'Ratios: HLG / ALL (Whole)': 'Ratio Mean'}, inplace = True)
      # rename column for better readbility

median_df = data_df.groupby(['gene']).median()                  # find the median as an independent dataframe
median_df.rename(columns = {'Ratios: HLG / ALL (Whole)': 'Ratio Median'}, inplace = True)
      # rename column for better readbility

count_df = data_df.groupby(['gene']).count()                    # total number of counts per gene as an independent dataframe
count_df.rename(columns = {'Ratios: HLG / ALL (Whole)': 'Count'}, inplace = True)
      # rename column for better readbility

result_df = data_df.reset_index(drop = True)                       # reset the index of the dataframe
result_df = pd.concat([mean_df, median_df, count_df], axis = 1)       
      # concatenate the independent dataframes of 'mean', 'median', and 'count' as one large dataframe

result_df = result_df.sort_values(by = ['Ratio Median'], ascending = False)
      # sort the concatenated dataframe by the median (greatest ⟶ least)

counter = float(input("Enter the count number that you want results that are less than to be cut off at: ")) 
      # asking user for input, stored as 'counter'
      # asking for a cutoff of the total number of counts for a gene
greater = result_df[result_df['Count'] < counter].index
      # independent dataframe from 'counter', where if the column 'Count' is less than the 'counter', it will be stored in 'greater'

result_df.drop(greater, inplace = True)
      # remove 'greater' from the concateneated dataframe that is sorted by the median value
result_df.reset_index(inplace = True)                        # reset the index of the dataframe

groupby = result_df.rename(columns = {'gene' : "Gene"})
      # new dataframe, using data from the dropped 'greater' and rename the column 'gene' to 'Gene'

data_lib = pd.read_csv('large-lib_full_2016.10.25.csv')
      # read in data from large-lib_full_2016.10.25.csv, stored in 'data_lib'

data_lib = data_lib.drop(['well', 'side', 'chromosome', 'plate', 'strand', 
                  'min_position', 'full_position', 'orientation', 
                  'gene_end_distances', 'total_reads', 
                  'if_both_sides', 'LEAPseq_distance', 
                  'LEAPseq_percent_confirming',
                  'LEAPseq_N_confirming', 'LEAPseq_N_nonconf', 
                  'LEAPseq_N_unique_pos',
                  'flanking_seq', 'plate_errors', 'well_errors', 
                  'plate_rpm', 'well_rpm',
                  'if_fixed_position'], axis = 1)
      # drop all ↑ columns from data_lib

level = float(input("Enter the confidence level that you want the data to be less than or equal to: "))
      # asking for user input, only accepting numerical values, for the confindence level that you want the resulting data to be less than or equal to, stored in 'level'
filter_data_lib = data_lib[data_lib['confidence_level'] <= level]
      # begin to filter data, using 'level' store the genes that are less than or equal to the confindence level entered for 'level' in 'filter_data_lib'
filter_data_lib = filter_data_lib.sort_values('confidence_level')
      # sort filter_data_lib by their confidence levels (least ⟶ greatest)
filter_data_lib = filter_data_lib.rename(columns = {"gene" : "Gene",
                                          "confidence_level" : "Confidence Level",
                                          "feature" : "Feature"})
      # rename columns: 'gene' to 'Gene', 'confidence_level' to "Confidence Level", and 'feature' to "Feature" 
filter_data_lib = filter_data_lib.reset_index(drop = True)
      # reset index, and drop old index

gene_name = filter_data_lib["Gene"].astype(str).str.replace(r'.v5.5$', '')
      # convert data type of column Gene to a string and replace '.v5.5$' with a whitespace
gene_name = gene_name.str.replace(r".v5.5", '').str.strip()
      # remove '.v5.5$'

filter_data_lib = filter_data_lib.drop(columns = {"Gene"})
      # drop Gene column
filter_data_lib["Gene"] = gene_name 
      # replace Gene Column by data where we removed '.v5.5$'

filter_data_lib =  filter_data_lib[["Gene", "Confidence Level", "IB", "Feature"]]
      # filter data, so that only columns Gene, Confidence Level, IB, and Feature remain and is stored into 'filter_data_lib'

df1 = groupby
      # dataframe1 (df1) is holding data from groupby, see above ↑

df3 = filter_data_lib
      # dataframe3 (df3) is holding data from filter_data_lib, see above ↑

leftBigMerge = pd.merge(left = df1, right = df3, how = 'left', 
                        left_on = 'Gene', right_on = 'Gene')
      # Merge both df1 and df3 to be one dataframe, aligned by gene names, named 'leftBigMerge'
      # the 'left' stuff is just so that we don't lose any data when merging, and just in case some data is double counted 
      # take it like a common factor, but the commonality (gene name) is on the left of the dataframe

leftBigMerge = leftBigMerge[["Gene", "Ratio Median", "Ratio Mean", 
                             "Feature", "Confidence Level", "IB"]]
      # leftBigMerge only to keep columns: Gene, Ratio Median, Ratio Mean, Feature, Confidence Level, and IB

# Now the data that we need from 2 dataframes has been consolidated into 1 dataframe, sorted by gene name, organized by ratio median

# Fishing 🐟🐠

**Fisher Exact Test Preparation**

In [ ]:
matrix = leftBigMerge                                     # set 'matrix' to be 'leftBigMerge'

oarfish = pd.DataFrame(matrix, columns = ["Gene", "Ratio Median",
                                          "Ratio Mean",
                                          "Description"])
      # create another dataframe, named 'oarfish' from data from 'matrix', taking on the columns listed
oarfish = oarfish.drop_duplicates()
      # drop duplicates from 'oarfish'

data = pd.read_csv('TAG Data.csv')
      # read in 'TAG Data.csv' and store the data in 'data'

data = data[['IB' , 'gene' , 'R5-S#11-HLG_WHOLE_normalized_reads' , 'R5-S#11-ALL_WHOLE_normalized_reads']]
      # have data only contain these columns from 'TAG Data.csv'
data ['Ratios: HLG / ALL (Whole)'] = data ['R5-S#11-HLG_WHOLE_normalized_reads'] / data['R5-S#11-ALL_WHOLE_normalized_reads']
      # create another column for the ratio, high lipid gate vs all

data = data.sort_values('gene')                           # sort 'data' by the gene names, alphabetically (numerically in this case)
data = data.replace([np.inf, -np.inf], np.nan)            # replace gene names that are 'np.inf' or '-np.inf' to 'np.nan'
data = data.dropna()                                      # drop all rows that have 'np.nan' which we coverted in the step above
data = data.drop(columns = {"R5-S#11-HLG_WHOLE_normalized_reads", "R5-S#11-ALL_WHOLE_normalized_reads"})
      # drop these columns because the only use for them was to create the ratio, so they are no longer needed

ratio = float(input("Enter value that you want data to be greater than (Ratio: HLG / ALL (Whole)): "))
      # requesting user input, stored as 'ratio', need to enter a float
      # asking for the value of ratio you want the remaining data to be greater than
data["RatioType"] = np.where(data["Ratios: HLG / ALL (Whole)"] > ratio, "Hit", "Non-Hit")
      # new column in 'data', entitled, "RatioType", where the ratio is greater than the ratio is greater than what was just entered
data = data.sort_values("gene")                           # sort 'data' by the gene names, alphabetically (numerically in this case)
data = data.reset_index(drop = True)                      # reset the index
data = data.replace(['unknown_chrom', 'no_gene_found',   
                     'gene_unknown'], np.nan)
      # replace values that are 'unknown_chrom', 'no_gene_found', or 'gene_unknown' to NaN
data = data.dropna()
      # drop all NaN values

unknown = data[data.gene.eq("unknown_chrom") |
               data.gene.eq("no_gene_found") |
               data.gene.eq("gene_unknown")]
      # create a dataframe where the gene name is 'unknown_chrom', 'no_gene_found', or 'gene_unknown'
      # what we previously dropped

unknown = unknown.reset_index(drop = True)
      # reset the index of 'unknown'

for gene, data_gene in data.groupby('gene'):
    data_gene
      # consolidates data to its IB, gene name, the ratio, and the ratio type
      # runs in a loop until entire dataframe has been searched     

**Actual Fisher Exact Test**

In [ ]:
bigHit = 0                                                 # initialize a variable, 'bigHit' and set to 0
bigNo = 0                                                  # initialize a varaible, 'bigNo' and set to 0

lastGene = data_gene['gene'].iloc[-1]                      # get the very last gene and its information from data_gene, stored as 'lastGene'
count = float(input("For the total number of counts (hits + non-hits) of a gene, enter a number for cutoff : "))
      # ask user for input, stored as floating 'count'
      # asking for the cutoff for genes that have a count that is less than 'count'

for gene, data_genie in data.groupby('gene'):               # starting of a loop, that will run and increment until the end of 'data'
    data_genie = data_genie.drop(columns = {'IB', "Ratios: HLG / ALL (Whole)"})
      # 'data_genie' is a dataframe that takes in every gene, one by one, per increment, and only keeps gene name, and ratio type 
   
    hit = data_genie.loc[data_genie.RatioType == 'Hit', "RatioType"].count()
      # hit is a dataframe that holds the genes with the ratio type, "Hit", and counts it
    noHit = data_genie.loc[data_genie.RatioType == "Non-Hit", "RatioType"].count()
      # noHit is a dataframe that holds the genes with the ratio type, "Non-Hit", and counts it

    bigHit =  bigHit + hit                                  # count the number of hits, adding bigHit on top of bigHit, which is why bigHit was initialized outside of the loop
    bigNo = bigNo + noHit                                   # count the number of non-hits, adding bigNo on top of bigNo, which is why bigNo was initialized outside of the loop

    if lastGene in data_genie.values:                       # if the lastGene is the current iteration, the loop ends, else, the current gene is not the lastGene, the loop continues to run
        sum = pd.DataFrame (                                # dataframe, 'sum', stores the total number of hits and total number of non-hits
            { 
                'Hit' : bigHit,                             # make a column, in 'sum', 'Hit' with the data from bigHit 
                'Non-Hit' : bigNo                           # make a column, in 'sum', 'Non-Hit' with the data from bigNo
            }, index = [0]                                
        )                                                   # end of if statement

# end of first for loop, now we have a dataframe with just the gene name (with their corresponsding ratio type), and the total number of hits and non-hits of all genes

goaway = pd.DataFrame()                                     # initialize empty dataframe, 'goaway'
biggergoaway = pd.DataFrame()                               # initialize empty dataframe, 'biggergoaway'

goname = pd.DataFrame()                                     # initialize empty dataframe, 'goname'
biggergoname = pd.DataFrame()                               # initialize empty dataframe, 'biggergoname'

for gene, data_genie in data.groupby('gene'):               # second for loop, will run and increment until the end of 'data'
    data_genie = data_genie.drop(columns = {'IB', "Ratios: HLG / ALL (Whole)"})
      # 'data_genie' is a dataframe that takes in every gene, one by one, per increment, and only keeps gene name, and ratio type
   
    hit = data_genie.loc[data_genie.RatioType == 'Hit', "RatioType"].count()
      # hit is a dataframe that holds the genes with the ratio type, "Hit", and counts it
    noHit = data_genie.loc[data_genie.RatioType == "Non-Hit", "RatioType"].count()
      # noHit is a dataframe that holds the genes with the ratio type, "Non-Hit", and counts it
    
    bigHit =  bigHit + hit                                  # count the number of hits, adding bigHit on top of bigHit, which is why bigHit was initialized outside of the loop
    bigNo = bigNo + noHit                                   # count the number of non-hits, adding bigNo on top of bigNo, which is why bigNo was initialized outside of the loop
    
    genius = pd.DataFrame (                                 # create a new dataframe, 'genius', with one column, 'Gene', storing each unique gene
        {
        'Gene': data_genie.gene.unique()
        }
    )
    
    frame = pd.DataFrame (                                  # create a new dataframe, 'frame', for each gene and their hits and non-hits
         {
            'Hit' : hit,
            'Non-Hit' : noHit
        }, index = [0]
    )

    concat = pd.concat([frame, sum], ignore_index = True)   # concatenate the dataframes 'frame' and 'sum' (individual gene hits and non-hits vs. total gene hits and non-hits) to 'concat'
    concat = concat.to_numpy()                              # convert 'concat' to all numpy values
    
    obs = np.array(concat)                                  # 'obs' takes on 'concat' as an array of numpy values instead of a dataframe
    new = pd.DataFrame(obs)                                 # 'new' takes on 'obs' as a dataframe

# beginning of the Fisher Exact Test, everything up until now was in preparation
    oddsratio, fisher = stats.fisher_exact(new)             # performs Fisher Exact Test on a 2 x 2 contingency table
        # contingency table:                                # store the p-values as 'fisher'
        # |        |   Hit   |   Non-Hit   |
        # |————————|—————————|—————————————|  
        # |  gene  |         |             | ← (this is from 'frame')
        # |————————|—————————|—————————————|  
        # |  total |         |             | ← (this is from 'sum')
        # this is what was concatenated a few lines above
    
    fishy = np.asarray(fisher)                              # convert 'fisher' to an array, stored as 'fishy' 
    fishy = fishy.reshape(-1, 1)                            
        # reshape p-values in 'fishy' to be their own arrays:
        # array ([[ p-value of the first gene],
        #         [ p-value of the second gene],
        #         [ p-value of the third gene], 
        #         etc.])                           

    frame.astype(float)                                     # ensure that 'frame' stores floats
    sum.astype(float)                                       # ensure that 'sum' stores floats

    fsh = pd.DataFrame(data = fishy, columns = ["p-value"])
        # 'fsh' is a dataframe storing the data from 'fishy' under a column "p-values"
    fsh.astype(float)                                       # make sure 'fsh' stores floats
    
    oarfish = np.concatenate((frame, sum), axis = 0)        # concatenate 'frame' and 'sum', store as 'oarfish'
    oarfish = pd.DataFrame(data = oarfish, columns = ["Hit", "Non-Hit"])
        # for the data in 'oarfish', title the two columns "Hit" and "Non-Hit"
    anchovy = pd.DataFrame(data = fsh, columns = ["p-value"])
        # data in 'fsh' is the p-values, stored as a dataframe, 'anchovy'

        # ↓ if I recall correctly, this is what is made for every gene ↓
        # | Gene Name                      |
        # |————————————————————————————————|
        # |        |   Hit   |   Non-Hit   |
        # |————————|—————————|—————————————|
        # |  gene  |         |             |
        # |————————|—————————|—————————————|  
        # |  total |         |             |
        # |————————————————————————————————|
        # |        |         |     p-value |

    bigFish = pd.concat([oarfish, anchovy], ignore_index = True, sort = True)
        # concatenate 'oarfish' and 'anchovy', stored as 'bigFish'
    bigFish = pd.concat([genius, bigFish], axis = 1)
        # update 'bigFish' by storing the concatenation of 'genius' (which was the unique gene names) 
        # and 'bigFish' (from the step above, is the concatenation of 'oarfish' and 'anchovy')

    bigFish.loc[bigFish['Hit'] + bigFish['Non-Hit'] < count] = "Go Away"
        # taking the user input, from the beginning of the cell, asking for the count that you want 
        # the remaining data to be greater than, from 'bigFish', replace the values 
        # in the gene name column with "Go Away"

    if bigFish.iloc[0, 0] == "Go Away":                       # if the gene name is exactly "Go Away"
      bigFish.iloc[2, 3] = "NO"                               # then "NO" is stored in the p-value's cell of the dataframe

    goaway = bigFish.iloc[2]                                  # 'goaway' stores the data from 'bigFish' if it is "Go Away"    
    biggergoaway = biggergoaway.append(goaway)                # 'biggergoaway' is used to append 'goway' to each other 

    goname = bigFish.iloc[0]                                  # 'goname' stores names from 'bigFish'
    biggergoname = biggergoname.append(goname)                # 'biggergoname' is used to append 'goname' to each other

# this all happens in a for loop, so the resulting dataframes are larger and larger
# please forgive the long run time

In [ ]:
# taking the last dataframe from the last iteration of the for-loop in the previous cell:
away = biggergoaway.drop(columns = ["Gene", "Hit", "Non-Hit"]) 
      # 'away' stores the data from 'biggergoaway', dropping the "Gene", "Hit", and "Non-Hit" columns
      # only "p-values" remains
away = away[away["p-value"] != "NO"]
      # 'away' is updated as it checks all the values in "p-value", keeping only those that are not "NO"
away = away.reset_index(drop = True)
      # reset the index of 'away'

pen = biggergoname.drop(columns = ["Hit", "Non-Hit", "p-value"])
      # 'pen' stores the data from 'biggergoname', dropping the "Hit", "Non-Hit", and "p-value" columns
      # only "Gene" remains
pen = pen[pen["Gene"] != "Go Away"]
      # 'pen' is updated as it checks all the values in "Gene", keeping only those that are not "Go Away"
pen = pen.reset_index(drop = True)
      # reset the index of pen

pen_away = pen.merge(away, left_index = True, right_index = True)
      # merge 'pen' and 'away' by the gene name, stored as 'pen_away'

#Benjamini-Hochberg Adjustment

In [ ]:
penguin = pen_away[~pen_away.Gene.str.contains(" & ")]
      # drop any rows in 'pen_away' with values in the "Gene" column that contain " & "
      # store the remaining rows in 'penguin'
penguin = penguin.reset_index(drop = True)                # reset the index of 'penguin'

bh_num = float(input("Enter the value that all adjusted p-values should be less than (0-1): "))
      # user input, asking for the maximum of the adjusted p-value, store as 'bh_num'

# ↓ Benjamini-Hochberg Adjustment of p-values ↓
# calls on a statisical adjustment from an R library: 
def FDR_adjust_pvalues(case, N=None, method='BH'):
    """ Adjust a list of p-values for false discovery rate using R's stats::p.adjust function.

    N and method are passed to R_stats.p_adjust:
     - N is the number of comparisons (if left unspecified, defaults to len(pvalue_list), I think)
     - method is the name of the adjustment method to use (inherited from R)

    Note that this MUST be done after all the p-values are already collected, on the full list of p-values at once:
     trying to do it on single p-values, even with adjusted N, will give different results!
    """
    if not method in R_stats.p_adjust_methods:
        raise ValueError("Unknown method %s - method must be one of (%s)!"%(method, ', '.join(R_stats.p_adjust_methods)))
    if N is None:   return R_stats.p_adjust(FloatVector(case), method=method)
    else:           return R_stats.p_adjust(FloatVector(case), method=method, n=N)

penguin["Adjusted p-value"] = FDR_adjust_pvalues(penguin['p-value'])
      # make a new column in 'penguin', "Adjusted p-value"
      # using False Discovery Rates (FDR) / correction on the adjusted p-values

penguin.sort_values(by = 'p-value')
      # sort 'penguin' by their p-values

adelie = penguin.loc[penguin["Adjusted p-value"] < bh_num]
      # keeps the rows with adjusted p-values that are less than 'bh_num', stored as dataframe 'adelie'

display(adelie)                                             # display 'adelie' as a dataframe

# conclusion: total time to run all cells 8 - 10 minutes
# majority of the time is taken by the "Actual Fisher Test" cell